In [16]:
import numpy as np

from jax.scipy.stats import norm
from types import SimpleNamespace

Environment is noiseless here.

In [34]:
class StaticEnvironment:
    def __init__(self, params: dict) -> None:
        self.params = SimpleNamespace(**params)
    
    def ge(self, x_star: float) -> float:
        return self.params.beta_1_star * x + self.params.beta_0_star

In [35]:
class StaticAgent:
    def __init__(self, params: dict) -> None:
        self.params = SimpleNamespace(**params)
        
    def gm(self):
        return self.params.beta_1 * self.params.x_range + self.params.beta_0
        
    def generative_model(self, y: float):
        likelihood = norm.pdf(y, loc=self.gm(), scale=self.params.std_y)
        prior      = norm.pdf(x_range, loc=self.params.mu_x, scale=self.params.std_x)
        return likelihood * prior
        
    def infer_state(self, y: float):
        gen_model = self.generative_model(y)
        evidence  = np.sum(gen_model, axis=0)
        posterior = gen_model / evidence
        return posterior
    

In [39]:
# Environment parameters
env_params = {
    "beta_0_star" : 3,    # Linear parameter intercept
    "beta_1_star" : 2,    # Linear parameter slope
}

# Agent parameters
agent_params = {
    "beta_0" : 3,    # Linear parameter intercept
    "beta_1" : 2,    # Linear parameter slope
    "std_y"  : 0.5,  # Likelihood standard deviation
    "mu_x"   : 4,    # Prior mean
    "std_x"  : 0.5,  # Prior standard deviation
    "x_range": np.linspace(start=0.01, stop=5, num=500)
}

In [43]:
# Initialize environment and agent
env       = StaticEnvironment(env_params)
agent     = StaticAgent(agent_params)

# Set true state and generate sensory data
x_star    = 2
y         = env.ge(x_star)

# Perception: Agent infers belief over hidden state using sensory data
posterior = agent.infer_state(y)

In [44]:
mode = (np.argmax(posterior) + 1) / 100
print(f'The posterior mode is {mode}. This is the expected or inferred food size, the hidden state estimate, when the observed light intensity is 7.')

The posterior mode is 2.4000000953674316. This is the expected or inferred food size, the hidden state estimate, when the observed light intensity is 7.
